# Data Preparation for GuitarTab Project

This notebook prepares the Guitar Chords V2 dataset for training the Chord Prediction Model. It includes loading audio files, converting them into spectrograms, encoding labels, and creating TensorFlow datasets for efficient training.

## Table of Contents
1. [Import Libraries](#Import-Libraries)
2. [Define Paths and Parameters](#Define-Paths-and-Parameters)
3. [Load and Encode Labels](#Load-and-Encode-Labels)
4. [Create TensorFlow Datasets](#Create-TensorFlow-Datasets)
5. [Visualize Sample Spectrograms](#Visualize-Sample-Spectrograms)
6. [Conclusion and Next Steps](#Conclusion-and-Next-Steps)

In [ ]:
# 1. Import Libraries
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from src.data_preprocessing import load_wav_16k_mono, preprocess_audio, compute_spectrogram, plot_waveform, plot_spectrogram, get_file_paths, encode_labels, preprocess_dataset
from sklearn.model_selection import train_test_split
import seaborn as sns
import pandas as pd

## 2. Define Paths and Parameters

Set the paths to your datasets and define any necessary parameters.

In [ ]:
# 2. Define Paths and Parameters
# Update these paths based on your local setup
DATA_DIR = '/content/drive/MyDrive/GuitarTab/data/raw/GuitarChordsV2/'
TRAIN_DIR = os.path.join(DATA_DIR, 'Training')
TEST_DIR = os.path.join(DATA_DIR, 'Test')

# Example chords (modify based on your dataset)
chords = ['Am', 'Bb', 'Bdim', 'C', 'Dm', 'Em', 'F', 'G']

## 3. Load and Encode Labels

Retrieve file paths and encode labels numerically and categorically.

In [ ]:
# 3. Load and Encode Labels
train_files, train_labels = get_file_paths(TRAIN_DIR, chords)
test_files, test_labels = get_file_paths(TEST_DIR, chords)

print(f"Number of training samples: {len(train_files)}")
print(f"Number of testing samples: {len(test_files)}")

# Encode labels
label_encoder, train_labels_cat = encode_labels(train_labels)
_, test_labels_cat = encode_labels(test_labels)  # Ensure same encoding

print(f"Encoded labels: {label_encoder.classes_}")

## 4. Create TensorFlow Datasets

Convert file paths and labels into TensorFlow datasets for efficient loading and preprocessing during training.

In [ ]:
# 4. Create TensorFlow Datasets
batch_size = 16
AUTOTUNE = tf.data.AUTOTUNE

# Create training and validation datasets
train_dataset = tf.data.Dataset.from_tensor_slices((train_files, train_labels_cat))
train_dataset = train_dataset.map(lambda x, y: preprocess_dataset([x.numpy()], [y.numpy()]),
                                num_parallel_calls=AUTOTUNE)
train_dataset = train_dataset.unbatch()
train_dataset = train_dataset.shuffle(buffer_size=1000).batch(batch_size).prefetch(AUTOTUNE)

validation_dataset = tf.data.Dataset.from_tensor_slices((test_files, test_labels_cat))
validation_dataset = validation_dataset.map(lambda x, y: preprocess_dataset([x.numpy()], [y.numpy()]),
                                          num_parallel_calls=AUTOTUNE)
validation_dataset = validation_dataset.unbatch()
validation_dataset = validation_dataset.batch(batch_size).prefetch(AUTOTUNE)

print("Training Dataset:")
for spectrogram, label in train_dataset.take(1):
    print(spectrogram.shape, label.shape)

print("Validation Dataset:")
for spectrogram, label in validation_dataset.take(1):
    print(spectrogram.shape, label.shape)

## 5. Visualize Sample Spectrograms

Visualize spectrograms to ensure data preprocessing is correct and to gain insights into the input data.

In [ ]:
# 5. Visualize Sample Spectrograms
import random

def visualize_samples(dataset, label_encoder, num_samples=5):
    class_names = label_encoder.classes_
    plt.figure(figsize=(15, num_samples * 3))
    for i, (spectrogram, label) in enumerate(dataset.take(num_samples)):
        spectrogram = spectrogram.numpy().squeeze()
        label = label.numpy()
        predicted_label = class_names[np.argmax(label)]
        plt.subplot(num_samples, 1, i+1)
        plt.imshow(spectrogram, aspect='auto', origin='lower', cmap='viridis')
        plt.title(f"Chord: {predicted_label}")
        plt.xlabel("Time Frame")
        plt.ylabel("Frequency Bin")
    plt.tight_layout()
    plt.show()

# Visualize 5 samples from training dataset
visualize_samples(train_dataset, label_encoder, num_samples=5)

## 6. Conclusion and Next Steps

- **Data Prepared**: Audio files have been loaded, preprocessed, and converted into spectrograms suitable for model training.
- **Datasets Created**: TensorFlow datasets for training and validation are ready.
- **Visual Verification**: Sample spectrograms have been visualized to ensure preprocessing correctness.

### **Next Steps**
- On Day 3, proceed to **Developing the Chord Prediction Model** by defining and training a CNN using the prepared datasets.